## 6-7. 프로젝트: 멋진 작사가 만들기
1) Step 1. 데이터 다운로드  
Song Lyrics 데이터를 다운  
  
2) Step 2. 데이터 읽어오기  
  
3) Step 3. 데이터 정제  
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!  
preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.  
  
추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래가사 작사하기에 어울리지 않을수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기를 권합니다.  
  
4) Step 4. 평가 데이터셋 분리  
tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!  
```
enc_train, enc_val, dec_train, dec_val = <코드 작성>

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


out:

Source Train: (124960, 14)
Target Train: (124960, 14)
```

  
5) Step 5. 인공지능 만들기  
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)  
``` 
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
    
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

```

***

평가문항
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?  
텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?  
  
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?  
특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?  
  
3. 텍스트 생성모델이 안정적으로 학습되었는가?   
텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?   



1) Step 1. 데이터 다운로드  
Song Lyrics 데이터를 다운  
  
2) Step 2. 데이터 읽어오기  

In [1]:
import glob # 파일을 읽어오는 작업을 하기가 아주 용이
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' # * = all : 모두 읽는다.

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['How does a bastard, orphan, son of a whore', 'And a Scotsman, dropped in the middle of a forgotten spot in the Caribbean by providence impoverished,', 'In squalor, grow up to be a hero and a scholar? The ten-dollar founding father without a father']


3) Step 3. 데이터 정제  
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!  
preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다. 

In [12]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(sentence)

this is sample sentence .
In the straw-colored light


In [3]:
corpus = []

for sentence in raw_corpus: # 
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
print("데이터 크기:", len(raw_corpus))
corpus[:10]

데이터 크기: 187088


['how does a bastard , orphan , son of a whore',
 'and a scotsman , dropped in the middle of a forgotten spot in the caribbean by providence impoverished ,',
 'in squalor , grow up to be a hero and a scholar ? the ten dollar founding father without a father',
 'got a lot farther by working a lot harder',
 'by being a lot smarter by being a self starter',
 'by fourteen , they placed him in charge of a trading charter and every day while slaves were being slaughtered and carted away',
 'across the waves , he struggled and kept his guard up',
 'inside , he was longing for something to be a part of',
 'the brother was ready to beg , steal , borrow , or barter then a hurricane came , and devastation reigned',
 'our man saw his future drip , dripping down the drain']

**문장 토큰 개수를 12개로 조정**

In [4]:
import tensorflow as tf    # 대망의 텐서플로우!

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 12)  

    print(tensor,tokenizer, sep='\n')
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
len(tensor[1])

[[   76   624     7 ...     7  3291     0]
 [  879    17     7 ...  9558 11329     2]
 [    6     7  7653 ...   256     7   633]
 ...
 [  205     1    24 ...     0     0     0]
 [   74     5   313 ...     0     0     0]
 [   12     4  5473 ...     0     0     0]]


12

**자른 텐서에 \<start> 와 \<end> 붙이기**

In [ ]:
def preprocess_sentence_2(sentence):
    
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence
print(preprocess_sentence_2("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

In [ ]:
corpus = []

for sentence in raw_corpus: 
        
    corpus.append(preprocess_sentence_2(sentence))
        
corpus[1]

In [ ]:
corpus = []

for sentence in raw_corpus: # 
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        

corpus[:10]

4) Step 4. 평가 데이터셋 분리  
tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!   
  
```
enc_train, enc_val, dec_train, dec_val = <코드 작성>

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


out:

Source Train: (124960, 14)
Target Train: (124960, 14)
```


# ?
아니 6-6에서 직접 평가해야 한다고 했는데, train과 test의 역활이 뭐지..? 나눠서 뭐할라고?
=> validation 으로 split

In [ ]:
import tensorflow as tf    # 대망의 텐서플로우!

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen = 15, padding='post')  

    print(tensor,tokenizer, sep='\n')
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
tensor[1]

In [ ]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력
print(tensor[:5, :10])

In [ ]:
# 단어 사전이 어떻게 구축되었는지 아래와 같이 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 20: break

In [ ]:
# 생성된 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있게
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

print(type(dataset))
dataset

In [ ]:
from sklearn.model_selection import train_test_split

# train_test_split(arrays, test_size, train_size, random_state, shuffle, stratify)

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input, 
                                                          test_size=0.2, 
                                                          random_state=20)

In [ ]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

5) Step 5. 인공지능 만들기  
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)  
``` 
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
    
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

```

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) # Embedding 레이어
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# 모델의 최종 출력 텐서 shape
for src_sample, tgt_sample in dataset.take(1): break
# print(model(src_sample))

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

# history = model.fit(partial_x_train,
#                     partial_y_train,
#                     epochs=epochs,
#                     batch_size=512,
#                     validation_data=(x_val, y_val),
#                     verbose=1)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

In [ ]:
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

print(loss)

In [ ]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)